# In V3 of the code we were sending TTLs via 5 pin for Pellet intake(and pellet in well), Right poke, Left Poke, Rightwithpellet poke and Leftwithpellet poke, now I am trying to see whether we can easily send signals of pokes via the same pins
# note that V4 was trying to make Raspberry Pi autmatically run the code --  while we decided to use a touch screen -- so V4 is archived and we proceed with V3 of the code.

In [ ]:
#!/usr/bin/env python3

import os
import sys
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import traceback
import re

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define adjusted GPIO pins for each device 
gpio_pins_per_device = {
    'Port 1': {
        "LeftPoke": 17,        # GPIO17 (Physical Pin 11) - Left Poke and LeftWithPellet
        "RightPoke": 27,       # GPIO27 (Physical Pin 13) - Right Poke and RightWithPellet
        "Pellet": 22,          # GPIO22 (Physical Pin 15) - Pellet event
    },
    'Port 2': {
        "LeftPoke": 10,        # GPIO10 (Physical Pin 19) - Left Poke and LeftWithPellet
        "RightPoke": 9,        # GPIO9 (Physical Pin 21)  - Right Poke and RightWithPellet
        "Pellet": 11,          # GPIO11 (Physical Pin 23) - Pellet event
    },
    'Port 3': {
        "LeftPoke": 5,         # GPIO5 (Physical Pin 29)  - Left Poke and LeftWithPellet
        "RightPoke": 6,        # GPIO6 (Physical Pin 31)  - Right Poke and RightWithPellet
        "Pellet": 13,          # GPIO13 (Physical Pin 27) - Pellet event
    },
    'Port 4': {
        "LeftPoke": 19,        # GPIO19 (Physical Pin 33) - Left Poke and LeftWithPellet
        "RightPoke": 26,       # GPIO26 (Physical Pin 35) - Right Poke and RightWithPellet
        "Pellet": 20,          # GPIO20 (Physical Pin 37) - Pellet event
    },
}

# Set all pins as output and initially set them to LOW
for device_pins in gpio_pins_per_device.values():
    for pin in device_pins.values():
        GPIO.setup(pin, GPIO.OUT)
        GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well per port_identifier
pellet_in_well = {}  # Dictionary to keep track per port_identifier

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type, port_identifier, gpio_pins):
    global pellet_in_well
    with pellet_lock:
        if port_identifier not in pellet_in_well:
            pellet_in_well[port_identifier] = False
        if event_type == "Pellet":
            if pellet_in_well[port_identifier]:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
                logging.debug(f"{port_identifier}: Pellet taken, signal turned OFF.")
                pellet_in_well[port_identifier] = False  # Update state
                send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
            else:
                logging.debug(f"{port_identifier}: No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
            pellet_in_well[port_identifier] = True
            logging.debug(f"{port_identifier}: Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, port_identifier, gpio_pins):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    logging.info(f"[{timestamp}] [{port_identifier}] Processing event: {event_type}")

    if event_type == "Left" or event_type == "LeftWithPellet":
        # Combine Left and LeftWithPellet using same signal
        send_ttl_signal(gpio_pins["LeftPoke"])
        logging.debug(f"{port_identifier}: Left poke event (with or without pellet) triggered.")

    elif event_type == "Right" or event_type == "RightWithPellet":
        # Combine Right and RightWithPellet using same signal
        send_ttl_signal(gpio_pins["RightPoke"])
        logging.debug(f"{port_identifier}: Right poke event (with or without pellet) triggered.")

    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type, port_identifier, gpio_pins)  # Handle PelletInWell and PelletTaken

# Function to get device mappings based on physical USB ports
def get_device_mappings_by_usb_port():
    device_mappings = []

    # Define the mapping from USB port paths to port identifiers
    usb_port_mapping = {
        'usb-0:1.1': 'Port 1',
        'usb-0:1.2': 'Port 2',
        'usb-0:1.3': 'Port 3',
        'usb-0:1.4': 'Port 4',
        # Add more mappings if you have more USB ports
    }

    # List all symlinks in /dev/serial/by-path/
    for symlink in os.listdir('/dev/serial/by-path/'):
        symlink_path = os.path.join('/dev/serial/by-path/', symlink)
        serial_port = os.path.realpath(symlink_path)
        if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
            logging.debug(f"Found serial port {serial_port} with symlink {symlink}")

            # Extract the USB port path from the symlink
            usb_port_path = get_usb_port_path_from_symlink(symlink)
            if not usb_port_path:
                continue

            port_identifier = usb_port_mapping.get(usb_port_path)
            if port_identifier:
                device_mappings.append({
                    'serial_port': serial_port,
                    'port_identifier': port_identifier,
                })
                logging.info(f"Mapped serial port {serial_port} to {port_identifier} based on USB port {usb_port_path}")
            else:
                logging.warning(f"No mapping found for USB port {usb_port_path}")
        else:
            logging.warning(f"Symlink {symlink} does not point to a recognized serial port.")
    return device_mappings

def get_usb_port_path_from_symlink(symlink):
    # Extract the USB port path from the symlink name
    # e.g., 'platform-...-usb-0:1.1:1.0' -> 'usb-0:1.1'
    match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
    if match:
        usb_port_path = match.group()
        return usb_port_path
    else:
        logging.warning(f"Could not extract USB port path from symlink {symlink}")
        return None

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port, port_identifier, gpio_pins):
    try:
        ser = serial.Serial(serial_port, 115200, timeout=1)
        logging.info(f"Opened serial port {serial_port} for {port_identifier}")
    except serial.SerialException as e:
        logging.error(f"Could not open serial port {serial_port}: {e}")
        return
    except Exception as e:
        logging.error(f"Unexpected error opening serial port {serial_port}: {e}")
        return

    try:
        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    logging.debug(f"{port_identifier} raw data: {data_list}")
                    if len(data_list) >= 10:
                        event_type = data_list[9]
                        logging.debug(f"{port_identifier}: Extracted event type: {event_type}")
                        process_event(event_type, port_identifier, gpio_pins)
                    else:
                        logging.warning(f"{port_identifier}: Received incomplete data, skipping this line.")
                else:
                    # No data received; can implement a sleep or pass
                    pass
            except UnicodeDecodeError as e:
                logging.error(f"{port_identifier}: Decoding error on serial port {serial_port}: {e}")
            except Exception as e:
                logging.error(f"{port_identifier}: Error reading from serial port {serial_port}: {e}")
                logging.error(traceback.format_exc())
        ser.close()
        logging.info(f"Serial port {serial_port} for {port_identifier} closed.")
    except Exception as e:
        logging.error(f"Critical error in thread for serial port {serial_port}: {e}")
        logging.error(traceback.format_exc())
    finally:
        try:
            ser.close()
            logging.info(f"Serial port {serial_port} for {port_identifier} closed.")
        except Exception as e:
            logging.error(f"Error closing serial port {serial_port}: {e}")

# Main execution
if __name__ == "__main__":
    logging.info("Starting device mapping based on USB ports...")
    device_mappings = get_device_mappings_by_usb_port()

    if not device_mappings:
        logging.error("No FED3 devices found. Exiting.")
        sys.exit(1)

    threads = []

    # Start threads for each device
    for mapping in device_mappings:
        serial_port = mapping['serial_port']
        port_identifier = mapping['port_identifier']
        gpio_pins = gpio_pins_per_device.get(port_identifier)
        if not gpio_pins:
            logging.error(f"No GPIO pins defined for {port_identifier}")
            continue
        logging.info(f"Setting up {port_identifier} on {serial_port}")
        t = threading.Thread(target=read_from_fed, args=(serial_port, port_identifier, gpio_pins))
        t.start()
        threads.append(t)

    # Cleanup GPIO when script ends
    try:
        while True:
            time.sleep(1)  # Keep the script running
    except KeyboardInterrupt:
        logging.info("Interrupted by user.")
        stop_event.set()  # Signal threads to stop
    finally:
        # Wait for threads to finish
        for t in threads:
            t.join()
        logging.info("Cleaning up GPIO and exiting.")
        GPIO.cleanup()
